<a href="https://colab.research.google.com/github/2019ht12169/Thesis/blob/main/simpleBertImpl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
! python3.7 -m pip install tensorflow-gpu==1.15.0
import pandas as pd
from sklearn.model_selection import train_test_split

In [164]:
# check out what the data looks like before you get started
# look at the training data set
train_df = pd.read_csv('/content/gdrive/MyDrive/projects/data/dataFile_latest_train.csv',index_col=None)
train_df=train_df.drop(['review_date'],axis='columns')
train_df = train_df.dropna().reset_index(drop=True)

# look at the test data set
test_df = pd.read_csv('/content/gdrive/MyDrive/projects/data/dataFile_latest_test.csv',index_col=None)
test_df = test_df.dropna().reset_index(drop=True)

In [ ]:
train_df['label'].astype(int)
test_df['label'].astype(int)

In [168]:
bert_df = pd.DataFrame({
    'id': range(len(train_df)),
    'label': train_df['label'],
    'alpha': ['q']*train_df.shape[0],
    'text': train_df['text'].replace(r'\n', ' ', regex=True)
})

train_bert_df, dev_bert_df = train_test_split(bert_df, test_size=0.2)

In [169]:
print(len(train_bert_df))
print(len(dev_bert_df))

992095
248024


In [172]:
test_bert_df = pd.DataFrame({
    'id': range(len(test_df)),
    'text': test_df['text'].replace(r'\n', ' ', regex=True),
    'review_date': test_df['review_date']
})

In [173]:
train_bert_df.to_csv('/content/gdrive/MyDrive/projects/data/Data-BERT/train.tsv', sep='\t', index=False, header=False)
dev_bert_df.to_csv('/content/gdrive/MyDrive/projects/data/Data-BERT/dev.tsv', sep='\t', index=False, header=False)
test_bert_df.to_csv('/content/gdrive/MyDrive/projects/data/Data-BERT/test.tsv', sep='\t', index=False, header=True)

In [ ]:
! python /content/gdrive/MyDrive/projects/bert/run_classifier.py --task_name=cola --do_train=true --do_eval=true --data_dir=/content/gdrive/MyDrive/projects/data/Data-BERT --vocab_file=/content/gdrive/MyDrive/projects/uncased_L-12_H-768_A-12/vocab.txt --bert_config_file=/content/gdrive/MyDrive/projects/uncased_L-12_H-768_A-12/bert_config.json --init_checkpoint=/content/gdrive/MyDrive/projects/uncased_L-12_H-768_A-12/bert_model.ckpt --max_seq_length=128 --train_batch_size=32 --learning_rate=2e-5 --num_train_epochs=3.0 --output_dir=/content/gdrive/MyDrive/projects/data/Data-BERT/model_output --do_lower_case=true


In [ ]:
!python /content/gdrive/MyDrive/projects/bert/run_classifier.py --task_name=cola --do_predict=true --data_dir=/content/gdrive/MyDrive/projects/data/Data-BERT --vocab_file=/content/gdrive/MyDrive/projects/uncased_L-12_H-768_A-12/vocab.txt --bert_config_file=/content/gdrive/MyDrive/projects/uncased_L-12_H-768_A-12/bert_config.json --init_checkpoint=/content/gdrive/MyDrive/projects/model_output_latest/model.ckpt-43736 --max_seq_length=128 --output_dir=/content/gdrive/MyDrive/projects/data/Data-BERT/model_output_latest/

In [175]:
#read the original test data for the text and id
df_test = pd.read_csv('/content/gdrive/MyDrive/projects/data/Data-BERT/test.tsv', sep='\t',index_col=None)

#read the results data for the probabilities
df_result = pd.read_csv('/content/gdrive/MyDrive/projects/data/Data-BERT/model_output_latest/test_results.tsv', sep='\t',names=['neg', 'pos'])


In [176]:
#create a new dataframe
df_map_result = pd.DataFrame({
    'pred_label': df_result.idxmax(axis=1),
    'actual_label': test_df['label'],
    'review_date': df_test['review_date'],
    'text': df_test['text']})
#view sample rows of the newly created dataframe
df_map_result.to_csv('/content/gdrive/MyDrive/projects/data/Data-BERT/mapped_results.csv',header=True,index=False)


In [ ]:
# grouping for data visualization [grouping by label]
score = df_map_result['text'].groupby(df_map_result['pred_label'])
score.count()

In [ ]:
# grouping for data visualization [grouping by review date's year]
df_map_result['review_date'] = pd.to_datetime(df_map_result['review_date'])
score1 = df_map_result['text'].groupby(df_map_result['review_date'].dt.strftime('%Y'))
score1.count()